In [0]:
# Run this cell and select the kaggle.json file downloaded
# from the Kaggle account settings page.
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mikuns","key":"19ed252a7541a96e597034e7e08fb1d0"}'}

In [0]:
# Let's make sure the kaggle.json file is present.
!ls -lha kaggle.json

-rw-r--r-- 1 root root 62 Oct  9 18:36 kaggle.json


In [0]:
# Next, install the Kaggle API client.
!pip install -q kaggle

In [0]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 /content/.kaggle/kaggle.json

chmod: cannot access '/content/.kaggle/kaggle.json': No such file or directory


In [0]:
!ls

kaggle.json  sample_data


In [0]:
# Copy the stackoverflow data set locally.
!kaggle competitions download -c dsn2018selection

100% 4.41M/4.41M [00:00<00:00, 18.3MB/s]

  0% 0.00/2.24M [00:00<?, ?B/s]
100% 2.24M/2.24M [00:00<00:00, 262MB/s]
  0% 0.00/1.15M [00:00<?, ?B/s]
100% 1.15M/1.15M [00:00<00:00, 112MB/s]


In [0]:
!ls /content

kaggle.json  sample_data  SampleSubmission.zip	test.zip  train.zip


In [0]:
import pandas as pd

train = pd.read_csv("/content/train.zip")
test = pd.read_csv("/content/test.zip")
submit = pd.read_csv("/content/SampleSubmission.zip")
train.head(3)

,Viewers_ID,Joke_identifier,Response_ID,Rating
0,A1,Klint De Drunk Enugu 1,A1_Klint De Drunk Enugu 1,0.11
1,A1,Klint De Drunk Enugu 2,A1_Klint De Drunk Enugu 2,-4.64
2,A1,Klint De Drunk PH 1,A1_Klint De Drunk PH 1,-3.39


In [0]:
test.head()

,Viewers_ID,Joke_identifier,Response_ID
0,A1,Akpororo Lagos 1,A1_Akpororo Lagos 1
1,A1,Akpororo Abuja 1,A1_Akpororo Abuja 1
2,A1,Akpororo Abuja 5,A1_Akpororo Abuja 5
3,A1,I Go Dye Benin 1,A1_I Go Dye Benin 1
4,A1,I Go Dye Benin 3,A1_I Go Dye Benin 3


In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, classification_report, accuracy_score
from math import sqrt

import numpy as np
from sklearn                        import metrics, svm
from sklearn.linear_model           import LinearRegression
from sklearn.linear_model           import LogisticRegression
from sklearn.tree                   import DecisionTreeClassifier
from sklearn.neighbors              import KNeighborsClassifier
from sklearn.discriminant_analysis  import LinearDiscriminantAnalysis
from sklearn.naive_bayes            import GaussianNB
from sklearn.svm                    import SVC

import xgboost as xgb


%matplotlib inline
plt.style.use('fivethirtyeight') #style
from IPython.display import display

pd.set_option('display.max_columns', 500)
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory


In [0]:
sample_t = train
sample_y = test

In [0]:
sample_t[['Comedian','Location','Volume']] = sample_t['Joke_identifier'].str.rsplit(n=2, expand=True)
sample_y[['Comedian','Location','Volume']] = sample_y['Joke_identifier'].str.rsplit(n=2, expand=True)

sample_t.head()

,Viewers_ID,Joke_identifier,Response_ID,Rating,Comedian,Location,Volume
0,A1,Klint De Drunk Enugu 1,A1_Klint De Drunk Enugu 1,0.11,Klint De Drunk,Enugu,1
1,A1,Klint De Drunk Enugu 2,A1_Klint De Drunk Enugu 2,-4.64,Klint De Drunk,Enugu,2
2,A1,Klint De Drunk PH 1,A1_Klint De Drunk PH 1,-3.39,Klint De Drunk,PH,1
3,A1,Klint De Drunk PH 2,A1_Klint De Drunk PH 2,0.44,Klint De Drunk,PH,2
4,A1,Klint De Drunk Lagos 1,A1_Klint De Drunk Lagos 1,-4.83,Klint De Drunk,Lagos,1


In [0]:
sample_y.sample(10)

,Viewers_ID,Joke_identifier,Response_ID,Comedian,Location,Volume
414090,A8175,Okey Bakassi Lagos 4,A8175_Okey Bakassi Lagos 4,Okey Bakassi,Lagos,4
281163,A33289,Okey Bakassi Abuja 1,A33289_Okey Bakassi Abuja 1,Okey Bakassi,Abuja,1
302890,A35083,Saka Lagos 3,A35083_Saka Lagos 3,Saka,Lagos,3
144307,A22475,Helen Paul Abuja 2,A22475_Helen Paul Abuja 2,Helen Paul,Abuja,2
419146,A8618,Seyi Law Abuja 2,A8618_Seyi Law Abuja 2,Seyi Law,Abuja,2
307166,A35413,Gordons Lagos 1,A35413_Gordons Lagos 1,Gordons,Lagos,1
49369,A14432,Klint De Drunk Warri 1,A14432_Klint De Drunk Warri 1,Klint De Drunk,Warri,1
115372,A20180,Funny Bone Abuja 3,A20180_Funny Bone Abuja 3,Funny Bone,Abuja,3
335722,A37893,MisterIbu Lagos 4,A37893_MisterIbu Lagos 4,MisterIbu,Lagos,4
31718,A12602,Helen Paul Lagos 1,A12602_Helen Paul Lagos 1,Helen Paul,Lagos,1


In [0]:
sample_t.Comedian[sample_t.Comedian == 'AliBaba Aso'] = 'AliBaba'
sample_t.Location[sample_t.Location == 'Rock'] = 'Aso Rock'

sample_y.Comedian[sample_y.Comedian == 'AliBaba Aso'] = 'AliBaba'
sample_y.Location[sample_y.Location == 'Rock'] = 'Aso Rock'

sample_y.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Viewers_ID,Joke_identifier,Response_ID,Comedian,Location,Volume
0,A1,Akpororo Lagos 1,A1_Akpororo Lagos 1,Akpororo,Lagos,1
1,A1,Akpororo Abuja 1,A1_Akpororo Abuja 1,Akpororo,Abuja,1
2,A1,Akpororo Abuja 5,A1_Akpororo Abuja 5,Akpororo,Abuja,5
3,A1,I Go Dye Benin 1,A1_I Go Dye Benin 1,I Go Dye,Benin,1
4,A1,I Go Dye Benin 3,A1_I Go Dye Benin 3,I Go Dye,Benin,3


In [0]:
sample_t['Viewers_status'] = sample_t.Viewers_ID.map(sample_t.Viewers_ID.value_counts())
sample_y['Viewers_status'] = sample_y.Viewers_ID.map(sample_y.Viewers_ID.value_counts())
sample_t.sample(5)

,Viewers_ID,Joke_identifier,Response_ID,Rating,Comedian,Location,Volume,Viewers_status
543091,A5506,Klint De Drunk PH 2,A5506_Klint De Drunk PH 2,-0.66,Klint De Drunk,PH,2,26
586322,A8438,AliBaba Abuja 2,A8438_AliBaba Abuja 2,4.52,AliBaba,Abuja,2,11
136952,A18811,AY Lagos 1,A18811_AY Lagos 1,3.03,AY,Lagos,1,16
320732,A29165,Akpororo Abuja 2,A29165_Akpororo Abuja 2,-1.22,Akpororo,Abuja,2,22
23749,A11480,Klint De Drunk PH 2,A11480_Klint De Drunk PH 2,-3.02,Klint De Drunk,PH,2,6


In [0]:
sample_y['Viewers_status'].unique()

array([16,  6, 32, 33,  4,  5,  7,  3, 18, 34, 19, 12, 10, 11, 22,  9, 17,
        8, 21, 15, 23, 27, 28, 26, 14, 13, 24, 29, 30, 20,  2, 25, 31, 37,
       36, 35])

In [0]:
def expertise(stat):
  if stat <= 10:
    return 1
  if stat > 10 and stat <=20:
    return 2
  if stat > 20 and stat <=30:
    return 3
  if stat > 30:
    return 4

sample_t['Viewers_expertise'] = sample_t.Viewers_status.apply(expertise)
sample_y['Viewers_expertise'] = sample_y.Viewers_status.apply(expertise)
sample_t.sample(5)

,Viewers_ID,Joke_identifier,Response_ID,Rating,Comedian,Location,Volume,Viewers_status,Viewers_expertise
489855,A38887,Bovi Abuja 2,A38887_Bovi Abuja 2,0.52,Bovi,Abuja,2,51,4
221761,A23479,AliBaba Lagos 5,A23479_AliBaba Lagos 5,1.22,AliBaba,Lagos,5,20,2
447525,A36363,Gordons Lagos 2,A36363_Gordons Lagos 2,-0.78,Gordons,Lagos,2,19,2
94708,A16297,Akpororo Abuja 1,A16297_Akpororo Abuja 1,0.77,Akpororo,Abuja,1,27,3
419807,A34703,I Go Dye Lagos 1,A34703_I Go Dye Lagos 1,-3.56,I Go Dye,Lagos,1,43,4


In [0]:
revamped_df = sample_t.drop(['Joke_identifier','Response_ID','Viewers_status'], axis=1)
test_df = sample_y.drop(['Joke_identifier','Response_ID','Viewers_status'], axis=1)
revamped_df.head()

,Viewers_ID,Rating,Comedian,Location,Volume,Viewers_expertise
0,A1,0.11,Klint De Drunk,Enugu,1,3
1,A1,-4.64,Klint De Drunk,Enugu,2,3
2,A1,-3.39,Klint De Drunk,PH,1,3
3,A1,0.44,Klint De Drunk,PH,2,3
4,A1,-4.83,Klint De Drunk,Lagos,1,3


In [0]:
revamped_df['Location'] = revamped_df['Location'].str.strip().map({'Enugu':1, 'PH':2, 'Lagos':3, 'Warri':4, 'Aso Rock':5, 'Abuja':6, 'Owerri':1, 'Benin':8, 'Uyo':9, 'Ibadan':10})
revamped_df['Comedian'] = revamped_df['Comedian'].str.strip().map({'Klint De Drunk':1, 'AliBaba':2, 'BasketMouth':3, 'AY':4, 'Bovi':5, 'Akpororo':6, 'I Go Dye':7, 'Helen Paul':8, 'Seyi Law':9, 'Gordons':10, 'Saka':11, 'Okey Bakassi':12, 'Funny Bone':13, 'MisterIbu':11})

test_df['Location'] = test_df['Location'].str.strip().map({'Enugu':1, 'PH':2, 'Lagos':3, 'Warri':4, 'Aso Rock':5, 'Abuja':6, 'Owerri':1, 'Benin':8, 'Uyo':9, 'Ibadan':10})
test_df['Comedian'] = test_df['Comedian'].str.strip().map({'Klint De Drunk':1, 'AliBaba':2, 'BasketMouth':3, 'AY':4, 'Bovi':5, 'Akpororo':6, 'I Go Dye':7, 'Helen Paul':8, 'Seyi Law':9, 'Gordons':10, 'Saka':11, 'Okey Bakassi':12, 'Funny Bone':13, 'MisterIbu':11})

revamped_df.head()

,Viewers_ID,Rating,Comedian,Location,Volume,Viewers_expertise
0,A1,0.11,1,1,1,3
1,A1,-4.64,1,1,2,3
2,A1,-3.39,1,2,1,3
3,A1,0.44,1,2,2,3
4,A1,-4.83,1,3,1,3


In [0]:
test_df.head()

,Viewers_ID,Comedian,Location,Volume,Viewers_expertise
0,A1,6,3,1,2
1,A1,6,6,1,2
2,A1,6,6,5,2
3,A1,7,8,1,2
4,A1,7,8,3,2


In [0]:
revamped_df['Viewers_ID'] = revamped_df['Viewers_ID'].str[1:]
revamped_df['Viewers_ID'] = pd.to_numeric(revamped_df['Viewers_ID'], errors='coerce')

test_df['Viewers_ID'] = test_df['Viewers_ID'].str[1:]
test_df['Viewers_ID'] = pd.to_numeric(test_df['Viewers_ID'], errors='coerce')

revamped_df['Viewers_ID'].dtypes

dtype('int64')

In [0]:
revamped_df.head()

,Viewers_ID,Rating,Comedian,Location,Volume,Viewers_expertise
0,1,0.11,1,1,1,3
1,1,-4.64,1,1,2,3
2,1,-3.39,1,2,1,3
3,1,0.44,1,2,2,3
4,1,-4.83,1,3,1,3


In [0]:
# ##plotting the relation of potential vs overall_rating 
# import matplotlib.pyplot as plt
# plt.figure(figsize=(13,14))
# plt.scatter(revamped_df[revamped_df.Comedian == 1],revamped_df.Rating, c='b')
# plt.xlabel("Viewers_ID")
# plt.ylabel("rating")
# plt.show()

# revamped_df[revamped_df.Comedian == 1]

In [0]:
from math import ceil, floor
def float_round(num, places = 0, direction = floor):
    return direction(num * (10**places)) / float(10**places)

def xox(x_id, imax = revamped_df['Viewers_ID'].max()):
  return [float_round(((xx / imax) * 10), 3, round) for xx in [x_id]]
   
revamped_df['Viewers_ID'] = revamped_df['Viewers_ID'].apply(xox)

def xox_2(x_id, imax = test_df['Viewers_ID'].max()):
  return [float_round(((xx / imax) * 10), 3, round) for xx in [x_id]]
   
test_df['Viewers_ID'] = test_df['Viewers_ID'].apply(xox_2)


In [0]:
revamped_df['Viewers_ID'] = revamped_df['Viewers_ID'].str[0]
revamped_df['Viewers_ID'] = pd.to_numeric(revamped_df['Viewers_ID'], errors='coerce')

test_df['Viewers_ID'] = test_df['Viewers_ID'].str[0]
test_df['Viewers_ID'] = pd.to_numeric(test_df['Viewers_ID'], errors='coerce')
test_df.tail()

,Viewers_ID,Comedian,Location,Volume,Viewers_expertise
435868,2.447,13,3,1,1
435869,2.447,12,3,2,1
435870,2.447,12,6,3,1
435871,2.447,11,3,1,1
435872,2.447,11,3,4,1


In [0]:
#Test Data
pre_final_test = test_df

final_test = pre_final_test.apply(pd.to_numeric, errors='coerce')

final_test.head()

,Viewers_ID,Comedian,Location,Volume,Viewers_expertise
0,0.0,6,3,1,2
1,0.0,6,6,1,2
2,0.0,6,6,5,2
3,0.0,7,8,1,2
4,0.0,7,8,3,2


In [0]:
pd.DataFrame(final_test['Viewers_ID']).dtypes

Viewers_ID    float64
dtype: object

In [0]:
revamped_df['Volume'] = pd.to_numeric(revamped_df['Volume'], errors='coerce')
revamped_df.dtypes

Viewers_ID           float64
Rating               float64
Comedian               int64
Location               int64
Volume                 int64
Viewers_expertise      int64
dtype: object

In [0]:
pre_train_x = revamped_df.drop(['Rating'], axis = 1)

train_x = pre_train_x.apply(pd.to_numeric, errors='coerce')
train_y = revamped_df['Rating'].values.astype(float)


pd.DataFrame(train_y).isnull().any()

0    False
dtype: bool

In [0]:
#ONE HOT ENCODING
#---------------------
# use pandas get_dummies function to generate one hot encoding for each feature
# append the new feature to the existing set

#TRAINING DATA
ComedyClass   = pd.get_dummies(train_x['Comedian'], prefix='Comedian')
train_x       = pd.concat([train_x, ComedyClass], axis=1)
train_x       = train_x.drop(['Comedian'],axis=1)

LocationClass = pd.get_dummies(train_x['Location'], prefix='Location')
train_x       = pd.concat([train_x, LocationClass], axis=1)
train_x       = train_x.drop(['Location'],axis=1)

VolumeClass   = pd.get_dummies(train_x['Volume'], prefix='Volume')
train_x       = pd.concat([train_x, VolumeClass], axis=1)
train_x       = train_x.drop(['Volume'],axis=1)

Viewers_expert = pd.get_dummies(train_x['Viewers_expertise'], prefix='Viewers_expertise')
train_x        = pd.concat([train_x, Viewers_expert], axis=1)
train_x        = train_x.drop(['Viewers_expertise'],axis=1)

train_x.head()

,Viewers_ID,Comedian_1,Comedian_2,Comedian_3,Comedian_4,Comedian_5,Comedian_6,Comedian_7,Comedian_8,Comedian_9,Comedian_10,Comedian_11,Comedian_12,Comedian_13,Location_1,Location_2,Location_3,Location_4,Location_5,Location_6,Location_8,Location_9,Location_10,Volume_1,Volume_2,Volume_3,Volume_4,Volume_5,Viewers_expertise_1,Viewers_expertise_2,Viewers_expertise_3,Viewers_expertise_4
0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
1,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
2,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
3,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
4,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0


In [0]:
#TESTING DATA
ComedyClass_2    = pd.get_dummies(final_test['Comedian'], prefix='Comedian')
final_test       = pd.concat([final_test, ComedyClass_2], axis=1)
final_test       = final_test.drop(['Comedian'],axis=1)

LocationClass_2  = pd.get_dummies(final_test['Location'], prefix='Location')
final_test       = pd.concat([final_test, LocationClass_2], axis=1)
final_test       = final_test.drop(['Location'],axis=1)

VolumeClass_2    = pd.get_dummies(final_test['Volume'], prefix='Volume')
final_test       = pd.concat([final_test, VolumeClass_2], axis=1)
final_test       = final_test.drop(['Volume'],axis=1)

Viewers_expert   = pd.get_dummies(final_test['Viewers_expertise'], prefix='Viewers_expertise')
final_test       = pd.concat([final_test, Viewers_expert], axis=1)
final_test       = final_test.drop(['Viewers_expertise'],axis=1)

final_test.head()

,Viewers_ID,Comedian_1,Comedian_2,Comedian_3,Comedian_4,Comedian_5,Comedian_6,Comedian_7,Comedian_8,Comedian_9,Comedian_10,Comedian_11,Comedian_12,Comedian_13,Location_1,Location_2,Location_3,Location_4,Location_5,Location_6,Location_8,Location_9,Location_10,Volume_1,Volume_2,Volume_3,Volume_4,Volume_5,Viewers_expertise_1,Viewers_expertise_2,Viewers_expertise_3,Viewers_expertise_4
0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
1,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0
2,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0
3,0.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0
4,0.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0


In [0]:
train_y

array([ 0.11, -4.64, -3.39, ...,  1.03,  1.22,  1.28])

In [0]:
# lab_enc = preprocessing.LabelEncoder()
# train_y_2 = lab_enc.fit_transform(train_y)

x_train, x_test, y_train, y_test = train_test_split(train_x,train_y, test_size=0.3, random_state=42)

In [0]:
clf = LinearRegression()
clf.fit(x_train, y_train)
print("LinearRegression")
LiR_pred = clf.predict(x_test)
LiR_rms = sqrt(mean_squared_error(y_test, LiR_pred))
LiR_rms

LinearRegression


2.5609932971718194

In [0]:
from xgboost import XGBRegressor

In [0]:
my_model = XGBRegressor(
                 colsample_bytree=0.9,
                 gamma=0.0,
                 learning_rate=0.33,
                 max_depth=6,
                 min_child_weight=1.5,
                 n_estimators=7200,                                                                  
                 reg_alpha=0.9,
                 reg_lambda=0.6,
                 subsample=0.2,                 seed=42,
                 silent=1)
# my_model = XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,colsample_bytree=1, max_depth=7)

my_model.fit(x_train, y_train, early_stopping_rounds=30, eval_set=[(x_test, y_test)], verbose=False)

predictions = my_model.predict(x_test)

In [0]:
XGB_rms = sqrt(mean_squared_error(y_test, predictions))
XGB_rms

2.4658918630353073

In [0]:
result = my_model.predict(final_test)

In [0]:
import pickle

In [0]:
filename = 'finalized_model_2_4024.sav'
pickle.dump(my_model, open(filename, 'wb'))

In [0]:
# load the model from disk
# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)
# print(result)

In [0]:
submit.head()

,Response_ID,Rating
0,A1_Akpororo Lagos 1,1.0
1,A1_Akpororo Abuja 1,1.0
2,A1_Akpororo Abuja 5,1.0
3,A1_I Go Dye Benin 1,1.0
4,A1_I Go Dye Benin 3,1.0


In [0]:
submission = pd.DataFrame({
        "Response_ID": submit["Response_ID"],
        "Rating": result
    })
submission

,Rating,Response_ID
0,3.579359,A1_Akpororo Lagos 1
1,5.710098,A1_Akpororo Abuja 1
2,4.853787,A1_Akpororo Abuja 5
3,4.860869,A1_I Go Dye Benin 1
4,2.851663,A1_I Go Dye Benin 3
5,4.346045,A1_I Go Dye Uyo 1
6,4.325154,A1_I Go Dye Uyo 2
7,2.792288,A1_Seyi Law Abuja 3
8,3.796023,A1_Seyi Law Abuja 4
9,3.763130,A1_Gordons Lagos 2


In [0]:
submission.to_csv('dsn_2018_result_2_4021.csv', index=False, header=True)
print(submission.head())

     Rating          Response_ID
0  3.579359  A1_Akpororo Lagos 1
1  5.710098  A1_Akpororo Abuja 1
2  4.853787  A1_Akpororo Abuja 5
3  4.860869  A1_I Go Dye Benin 1
4  2.851663  A1_I Go Dye Benin 3


In [0]:
from google.colab import files

# with open('example.txt', 'w') as f:
#   f.write('some content')

files.download('dsn_2018_result_2_4021.csv')

In [0]:
# FinalResult=pd.DataFrame({'PassengerId':test_df["PassengerId"],'Survived':Y_Pred.astype(int)})
# FinalResult.to_csv('gender_submission.csv', index=False)
# TestResult=pd.DataFrame(test_df)

In [0]:
#XGBRegressor is for continuous target/outcome variables. These are often called "regression problems."

#XGBClassifier is for categorical target/outcome variables. These are often called "classification problems."

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import files
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# df.to_csv('mydataframe.csv', sep='\t')

# file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
# for file1 in file_list:
#   print('title: %s, id: %s' % (file1['title'], file1['id']))
  
file = drive.CreateFile({'parents':[{u'id': '1z9swU8JHkP5Xf-ycdft19e45vfbt7Pgd'}]}) 
file.SetContentFile("dsn_2018_result_2_4021.csv")
file.Upload()